In [1]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from Loan_APP.constants import *
from Loan_APP.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config

In [13]:
import os
from Loan_APP import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [14]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    # We can add all type of data transformation technique here. like PCA, Scaler, EDA
    
    def data_cleaning(self):
        data = pd.read_csv(self.config.data_path)
        data.drop('loan_id', axis =1 , inplace = True)
        
        nan_cat_feature = ['gender', 'married', 'self_employed', 'credit_history']
        for col in nan_cat_feature:
            data[col].fillna(data[col].mode()[0], inplace = True)
            
        nan_cont_feature = ['dependents', 'loanamount', 'loan_amount_term']
        for col in nan_cont_feature:
            data[col].fillna(data[col].median(), inplace = True)
            
        print(data.isnull().sum())
        
        df_obj = data.select_dtypes('O')
        
        lbl_ecoder = LabelEncoder()
        for col in df_obj:
            data[col] = lbl_ecoder.fit_transform(data[col])
        
        print(data.head())
        
        return data
        
    def train_test_spliting(self, data):
        # data = pd.read_csv(self.config.data_path)
        
        train, test = train_test_split(data, test_size=0.2, random_state= 42)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index = False)
        
        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        
        print(train.shape)
        print(test.shape)
    
    

In [15]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    cleaned_df = data_transformation.data_cleaning()
    data_transformation.train_test_spliting(cleaned_df)
except Exception as e:
    raise e

[2025-03-07 20:05:27,721: INFO: common: yaml file: config\config.yaml is loaded successfully!]
[2025-03-07 20:05:27,726: INFO: common: yaml file: params.yaml is loaded successfully!]
[2025-03-07 20:05:27,737: INFO: common: yaml file: schema.yaml is loaded successfully!]
[2025-03-07 20:05:27,742: INFO: common: created directory at: artifacts]
[2025-03-07 20:05:27,747: INFO: common: created directory at: artifacts/data_transformation]
gender               0
married              0
dependents           0
education            0
self_employed        0
applicantincome      0
coapplicantincome    0
loanamount           0
loan_amount_term     0
credit_history       0
property_area        0
loan_status          0
dtype: int64
   gender  married  dependents  ...  credit_history  property_area  loan_status
0       1        0         0.0  ...             1.0              2            1
1       1        1         1.0  ...             1.0              0            0
2       1        1         0.0  ..

C:\Users\shubh\AppData\Local\Temp\ipykernel_18652\768962064.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace = True)
C:\Users\shubh\AppData\Local\Temp\ipykernel_18652\768962064.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo